## Import libraries

In [1]:
import os
import json
import spotipy
import requests
import numpy as np
import pandas as pd
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import pairwise_distances, cosine_similarity

### I will read in the dataframe I have build my model on, and the one I use for grouping by tempo, and while I'm at it I will read in the dataframe of the names and the uris we had created in the modeling process so we can Identify the songs 
***doing so I will not have to tinker with the api and it's a HUGE detour***

### I will set all of the indexes as col=index

In [2]:
matrix = pd.read_csv('./Data/recommender.csv')
df= pd.read_csv('./Data/identifier.csv')
df_id= pd.read_csv('./Data/uri.csv')

In [3]:
df_id.set_index('index',inplace=True)
df_id.head(1)

,uri
index,
"siren 042, lala lala",spotify:track:75nN4kH1uzSsUdMrdUVfrq


In [4]:
df.set_index('index',inplace=True)
df.head(1)

,tempo,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,time_signature,duration_ms
index,,,,,,,,,,,,,
"siren 042, lala lala",-0.50495,0.45834,-0.112834,1.722145,0.482366,0.570691,-0.546041,1.386794,-0.389266,-0.545219,-0.481937,0.231248,-0.882392


In [5]:
matrix.set_index('index',inplace=True)

In [6]:
matrix.head()

,"siren 042, lala lala","make me a song, eleanor friedberger","uprising, muse","time is running out, muse","knights of cydonia, muse","vicious, lou reed","like a hurricane - 2017 remaster, neil young","smile like you mean it, the killers","season of the witch, donovan","harvest moon, neil young",...,"riders on the storm, the doors","you never can tell, chuck berry","you really got me - mono mix, the kinks","whole lotta love - 1990 remaster, led zeppelin","water under the bridge, adele","mannish boy, muddy waters","white rabbit, jefferson airplane","people are strange, the doors","valerie - live at bbc radio 1 live lounge, london / 2007, amy winehouse","fortunate son, creedence clearwater revival"
index,,,,,,,,,,,,,,,,,,,,,
"siren 042, lala lala",1.000000,-0.311383,-0.286814,0.111002,-0.083149,-0.241955,-0.187183,-0.350366,-0.262670,0.098482,...,0.020727,0.183704,0.161542,-0.207140,-0.121297,-0.297464,0.274764,0.217554,0.352860,-0.142514
"make me a song, eleanor friedberger",-0.311383,1.000000,0.539888,-0.031261,-0.197212,0.113686,0.152729,-0.226719,0.550992,0.431224,...,0.065870,-0.291463,-0.029947,-0.252953,0.061930,-0.036292,0.003406,-0.203497,0.045029,0.430267
"uprising, muse",-0.286814,0.539888,1.000000,0.336954,0.402456,0.204254,0.362574,0.533363,0.103877,-0.212970,...,-0.117869,-0.056616,0.371758,0.204847,0.419852,-0.034867,-0.366772,-0.499747,-0.290993,0.352033
"time is running out, muse",0.111002,-0.031261,0.336954,1.000000,0.744066,-0.070075,0.636314,0.583413,-0.294301,-0.497447,...,0.514783,-0.138062,0.340327,0.073770,0.870793,-0.355957,-0.435121,0.048796,0.188430,-0.187611
"knights of cydonia, muse",-0.083149,-0.197212,0.402456,0.744066,1.000000,-0.213141,0.744379,0.751676,-0.365332,-0.506441,...,0.488692,0.053141,0.215766,0.507136,0.587601,-0.138404,-0.568811,-0.339100,-0.177124,-0.522041


### I will once again request a token from Spoitify, this is just like the first time we requested a token, but this time we need to change the scoope to ***playlist-modify-private*** 
***make sure you set your token to a global variable so you can use it in the next step***

In [ ]:
def get_token(scope=None):
    
    redirect_uri = os.environ['SPOTIPY_REDIRECT_URI']
    username = os.environ['SPOTIPY_USERNAME']
    client_id = os.environ['SPOTIPY_CLIENT_ID']
    client_secret = os.environ['SPOTIPY_CLIENT_SECRET']
    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
    return token

token = get_token(scope='playlist-modify-private')
sp = spotipy.Spotify(auth=token)

In [ ]:
get_token()

### now I will run the function where I can get rcomendations based on a song, I will set my output to a set of uri's
***(set of uri's of the songs that are recommended based on the song of choice)***
### make sure you set your output to a global variable so we can use it to dump tracks in the playlist!

In [7]:
def play_list(choice= None):
    list_of_recoms =[]
    uri= []
    for song in df.loc[df.index.str.contains(choice)].sort_values(by='tempo', ascending = True).index:
        list_of_recoms.append(matrix[song].sort_values(ascending = False)[0:10])
        list_of_recoms.append(matrix[song].sort_values(ascending = False)[19:9:-1])        
        recom= pd.DataFrame(list_of_recoms).T
        recom= recom.drop(columns=choice)
        uri_df = pd.merge(recom,df_id, left_index=True, right_index=True)
        uris= uri_df.reset_index(drop=True)
        uris= uri_df.set_index('uri')
        uri.append(uris.index)
        return uri[0].values
    
    
uri_request= play_list(choice='harvest moon, neil young')

### now using the token we had recived from spotify we will make a playlist, we will recive a playlist Id which we need to set to a global variable, so we can access the playlist and dump music in it!

In [ ]:
def create_playlist(song= None):
    endpoint_url = f"https://api.spotify.com/v1/users/{os.environ['SPOTIPY_USERNAME']}/playlists"

    request_body = json.dumps({"name": f"playlist based on {song}",
                                "description": f"play list based on {song}",
                                "public": False })
    response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", 
                        "Authorization":f"Bearer {token}"})

    url = response.json()['external_urls']['spotify']
    playlist_id = response.json()['id']
    print(f'PLAYLIST RESPONSE STATUS: {response.status_code}')
    print(f'PLAYLIST ID: {playlist_id}')
    print(f'URL: {url}')
    return playlist_id
playlist_id_returned= create_playlist('whole lotta love - 1990 remaster, led zeppelin')

### lastly using the potipy library, we need to call on our username, the playlist_id and the list of recomended uri's that our recomender had build to dump music in our playlist! 

In [ ]:
sp.user_playlist_add_tracks(user=os.environ['SPOTIPY_USERNAME'], playlist_id=playlist_id_returned, tracks=uri_request)

# et voila!